In [ ]:
#Nome: Isabelle Ramos de Azevedo Ferreira
#RA: 10365077

import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import numpy as np
#Isabelle Ramos de Azevedo Ferreira
dados = pd.read_csv('dados.csv', sep=';', decimal=',')

dados.dropna(subset=['ANO CORTE', 'TOTAL ÁRVORES'], inplace=True)

anos = dados['ANO CORTE'].values.reshape(-1, 1)
total_arvores = dados['TOTAL ÁRVORES'].values.reshape(-1, 1)

X_treino, X_teste, y_treino, y_teste = train_test_split(anos, total_arvores, test_size=0.2, random_state=42)

# Criar um pipeline com características polinomiais, escalonamento e regressão ridge
pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=3)),
    ('scaler', StandardScaler()),
    ('ridge', Ridge())
])

# Configurar a grade de hiperparâmetros para o GridSearchCV
parametros = {
    'poly__degree': [2, 3, 4],
    'ridge__alpha': [0.1, 1.0, 10.0]
}

# Criar e treinar o modelo usando GridSearchCV
grid_search = GridSearchCV(pipeline, parametros, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_treino, y_treino)

# Exibir os melhores parâmetros encontrados pelo GridSearchCV
print("Melhores parâmetros encontrados: ", grid_search.best_params_)

y_pred = grid_search.predict(X_teste)
mse = mean_squared_error(y_teste, y_pred)
print(f"Erro Quadrático Médio no conjunto de teste: {mse:.2f}")

# Gerar previsões para anos futuros
anos_futuros = np.arange(2021, 2051).reshape(-1, 1)
anos_futuros_poly = grid_search.best_estimator_.named_steps['poly'].transform(anos_futuros)
anos_futuros_escalado = grid_search.best_estimator_.named_steps['scaler'].transform(anos_futuros_poly)
previsoes = grid_search.best_estimator_.named_steps['ridge'].predict(anos_futuros_escalado)

resultado = pd.DataFrame({'Ano': anos_futuros.flatten(), 'Total de Árvores': previsoes.flatten().round(2)})
resultado['Total de Árvores'] = resultado['Total de Árvores'].map('{:.2f}'.format)

print("\nPrevisão de Total de Árvores Cortadas para os Próximos Anos:")
print(resultado.to_string(index=False))
